# Problem Set 4



Team: 
    Lea Schmierer 3546563;
    Angelina Basova 3704658;
    Daniel Knorr 3727033

## Exercise 1


## Exercise 2

#### a) Explain briefly why all nodes in the clique have the same PageRank value.

All nodes in the clique have the same value, because they all have the same number of in-links and out-links, therefore none of these nodes are more important than the others. Every note is weighted the same.



#### b) 
You can find the solution of this exercise in the zip file as a pdf. 

#### c)
You can find the solution of this exercise in the zip file as a pdf. 


#### d) Explain briefly why random teleports not only solve the spider trap problem but also the dead ends problem. 

There are two approaches to dealing with dead ends: 
- We can drop the dead ends from the graph and their incoming arcs.
- We can modify the process by which random surfers are assumed to move about the web. This approach also resolves the problem of spider traps. It is also called taxation


Solution: 
The previous procedure describes how the columns of a matrix M were made stochastic so that teleportation can always be done when there is nowhere else to go.

vom BUCH:

"Note that if the graph has no dead ends, then the probability of introducing a
new random surfer is exactly equal to the probability that the random surfer will
decide not to follow a link from their current page. In this case, it is reasonable
to visualize the surfer as deciding either to follow a link or teleport to a random
page. However, if there are dead ends, then there is a third possibility, which
is that the surfer goes nowhere. Since the term (1 − )e/n does not depend on
the sum of the components of the vector v, there will always be some fraction
of a surfer operating on the Web. That is, when there are dead ends, the sum
of the components of v may be less than 1, but it will never reach 0."

## Exercise 3

In [75]:
import numpy as np


######################
# 3a)
print("Exercise 3a)")
######################


# return matrix A
def get_google_matrix(M, beta):
    scalar = np.ones(shape=M.shape) * 1 / M.shape[0]
    A = beta * M + (1-beta) * scalar 
    return A

def can_terminate(difference, epsilon):
    return np.all(difference < epsilon)


def calculatePageRank(M, beta, epsilon):
    pagerank = []

    A = get_google_matrix(M, beta)

    # init rank r
    num_nodes = M.shape[0]
    # r_0
    old_r = np.full((num_nodes, 1), 1/num_nodes)

    count = 0

    while True:
        new_r = np.dot(A, old_r)
        difference = np.abs(new_r - old_r)

        old_r = new_r

        count +=1

        # if termination criterion fulfilled stop power iteration
        if can_terminate(difference, epsilon):
            pagerank = new_r
            break

    print()
    print(f"Number of iterations: {count}")
    return pagerank


if __name__ == "__main__":
    matrix = np.matrix([[1/2, 1/2,0],[1/2, 0,0], [0, 1/2, 1]])
    print("Matrix M:")
    print(matrix)
    
    pagerank = calculatePageRank(matrix, 0.8, 0.001 )
    print()
    print("Pagerank results:")
    print(pagerank)
    
   

Exercise 3a)
Matrix M:
[[0.5 0.5 0. ]
 [0.5 0.  0. ]
 [0.  0.5 1. ]]

Number of iterations: 12

Pagerank results:
[[0.2130343 ]
 [0.15207948]
 [0.63488622]]


In [88]:
######################
# 3b)
print("Exercise 3b)")
######################

def generate_matrix(n:int):
    M = np.full((n, n), 1/(n-1))
    np.fill_diagonal(M, 0)

    return M


if __name__ == "__main__":
    print()
    M_4 = generate_matrix(4)
    print("Clique with 4 nodes")
    print(M_4)
    pagerank_4 = calculatePageRank(M_4, 0.8, 1/12)
    print("Pagerank")
    print(pagerank_4)

    print("-------------------------------------------------------------------")
    print("-------------------------------------------------------------------")


    print()
    print("Clique with 6 nodes")
    M_6 = generate_matrix(6)
    print(M_6)
    pagerank_6 = calculatePageRank(M_6, 0.8, 1/12)
    print("Pagerank")
    print(pagerank_4)


Exercise 3b)

Clique with 4 nodes
[[0.         0.33333333 0.33333333 0.33333333]
 [0.33333333 0.         0.33333333 0.33333333]
 [0.33333333 0.33333333 0.         0.33333333]
 [0.33333333 0.33333333 0.33333333 0.        ]]

Number of iterations: 1
Pagerank
[[0.25]
 [0.25]
 [0.25]
 [0.25]]
-------------------------------------------------------------------
-------------------------------------------------------------------

Clique with 6 nodes
[[0.  0.2 0.2 0.2 0.2 0.2]
 [0.2 0.  0.2 0.2 0.2 0.2]
 [0.2 0.2 0.  0.2 0.2 0.2]
 [0.2 0.2 0.2 0.  0.2 0.2]
 [0.2 0.2 0.2 0.2 0.  0.2]
 [0.2 0.2 0.2 0.2 0.2 0. ]]

Number of iterations: 1
Pagerank
[[0.25]
 [0.25]
 [0.25]
 [0.25]]


## Exercise 4

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType


######################
# 4a)
print("Exercise 4a)")
######################

sparkSession = SparkSession.builder.appName('A4E4').getOrCreate()

print("# --------------------------------")
print("# Create DataFrame DF1 webStanfordDF")
print("# --------------------------------")

rdd_datafile = sparkSession.sparkContext.textFile('./web-Stanford_small.txt')

rdd_web_standford = rdd_datafile.map(lambda x: x.split("\t"))

webStanfordRDD = rdd_web_standford.map(lambda p: (
    int(p[0]), int(p[1].strip())))

schema = StructType([
    StructField("FromNodeId", IntegerType(), True),
    StructField("ToNodeId", IntegerType(), True)
])

webStanfordDF = sparkSession.createDataFrame(webStanfordRDD, schema)

# Get all Nodes as a Set
listFromNodeId = webStanfordDF.select(webStanfordDF["FromNodeId"]).toPandas()['FromNodeId']
listeToNodeId = webStanfordDF.select(webStanfordDF["ToNodeId"]).toPandas()['ToNodeId']

listFromNodeId = list(listFromNodeId)
listeToNodeId = list(listeToNodeId)

allNodesList = listFromNodeId + listeToNodeId
allNodesSet = set(allNodesList)

# Define dictionary
dic = {}

def generateDictionary(allNodesSet, webStanfordDF, dic):
    for node in allNodesSet:
        liste = []
        filteredByFromNodeId = webStanfordDF.filter(webStanfordDF["FromNodeId"] == node)
        listOutgoingLinksTo = filteredByFromNodeId.select(filteredByFromNodeId["ToNodeId"]).toPandas()['ToNodeId']
        filteredByToNodeId = webStanfordDF.filter(webStanfordDF["ToNodeId"] == node)
        listIngoingLinksFrom = filteredByToNodeId.select(filteredByToNodeId["FromNodeId"]).toPandas()[
            'FromNodeId']
        listOutgoingLinksTo = list(listOutgoingLinksTo)
        listIngoingLinksFrom = list(listIngoingLinksFrom)
        liste.append(listOutgoingLinksTo)
        liste.append(listIngoingLinksFrom)
        dic[node] = liste


generateDictionary(allNodesSet, webStanfordDF, dic)

print("Dictionary Composition: Node i: <[Out-neighbors of i]>,<[In-neighbors of i]>")
print(dic)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType

sparkSession = SparkSession.builder.appName('A4E4').getOrCreate()

######################
# 4b)
print("Exercise 4b)")
######################

rdd_datafile = sparkSession.sparkContext.textFile('./web-Stanford_small.txt')

rdd_web_standford = rdd_datafile.map(lambda x: x.split("\t"))

schema = StructType([
    StructField("FromNodeId", StringType(), False),
    StructField("ToNodeId", StringType(), False)
])

webStanfordDF = sparkSession.createDataFrame(rdd_web_standford, schema)

#create FromNode_list
FromNode_list = webStanfordDF.rdd.map(lambda x: x.FromNodeId).collect()

# create ToNode_set
ToNode_list = webStanfordDF.rdd.map(lambda x: x.ToNodeId).collect()
ToNode_set = set(ToNode_list)

# die ganz Toten finden (die, die nicht woanders hinzeigen)
dead_ends = set()

for ToNode in ToNode_set:
    if ToNode not in FromNode_list:
        dead_ends.add(ToNode)

# die Toten finden, die nur auf die Toten zeigen
# python gesamtliste mit Pandas erzeugen
pdframe = webStanfordDF.toPandas()
gesamtliste = list(pdframe.values.tolist())
gesamtliste.sort()

neuToteSuchen = True
while neuToteSuchen:
    neue_dead_ends = set()
    FromNodeOld = ""
    countNotDead = 0
    for x in gesamtliste:
        FromNode = x[0]
        ToNode = x[1]
        if FromNode != FromNodeOld:  # gruppenwechsel
            if FromNodeOld != "":    # beim erstem mal nicht
                if countNotDead == 0:
                    dead_ends.add(FromNodeOld)
                    neue_dead_ends.add(FromNodeOld)
            FromNodeOld = FromNode
            countNotDead = 0
        if ToNode not in dead_ends:
            countNotDead = countNotDead + 1

    if len(neue_dead_ends) == 0:
        neuToteSuchen = False   # keine mehr gefunden - das wars

    workliste = [x for x in gesamtliste if x[0] not in neue_dead_ends]
    gesamtliste = workliste

print("Anzahl DEAD-ENDS: ", len(dead_ends))
print("Dead End Liste: ", dead_ends)